# Text Normalization (Chinese)

- `text_normalizer_zh.py`
- Including functions for:
    - word-seg chinese texts
    - clean up texts by removing duplicate spaces and line breaks
    - remove incompatible weird characters

In [1]:
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
import text_normalizer_zh as tnz

## Normalization Functions

In [ ]:
# %load text_normalizer_zh.py
"""

Notes
-----

    These functions are based on the text normalization functions 
    provided in Text Analytics with Python 2ed.

"""

import unicodedata
import re
# from nltk.tokenize.toktok import ToktokTokenizer
import pandas as pd
import jieba

## Initialize Trad Chinese dictionary
jieba.set_dictionary('../../../RepositoryData/data/jiaba/dict.txt.jiebatw.txt')


## Normalize unicode characters
def remove_weird_chars(text):
    #     ```
    #     (NFKD) will apply the compatibility decomposition, i.e.
    #     replace all compatibility characters with their equivalents.
    #     ```
    text = unicodedata.normalize('NFKD', text).encode('utf-8',
                                                      'ignore').decode(
                                                          'utf-8', 'ignore')
    return text


## Remove extra linebreaks
def remove_extra_linebreaks(text):
    lines = text.split(r'\n+')
    return '\n'.join(
        [re.sub(r'[\s]+', ' ', l).strip() for l in lines if len(l) != 0])


## Remove extra medial/trailing/leading spaces
def remove_extra_spaces(text):
    return re.sub("\\s+", " ", text).strip()


## Seg the text into words
def seg(text):
    text_seg = jieba.cut(text)
    out = ' '.join(text_seg)
    return out


## Remove punctuation/symbols
def remove_symbols(text):
    """
    
    Unicode 6.0 has 7 character categories, and each category has subcategories:

    Letter (L): lowercase (Ll), modifier (Lm), titlecase (Lt), uppercase (Lu), other (Lo)
    Mark (M): spacing combining (Mc), enclosing (Me), non-spacing (Mn)
    Number (N): decimal digit (Nd), letter (Nl), other (No)
    Punctuation (P): connector (Pc), dash (Pd), initial quote (Pi), final quote (Pf), open (Ps), close (Pe), other (Po)
    Symbol (S): currency (Sc), modifier (Sk), math (Sm), other (So)
    Separator (Z): line (Zl), paragraph (Zp), space (Zs)
    Other (C): control (Cc), format (Cf), not assigned (Cn), private use (Co), surrogate (Cs)
    
    
    There are 3 ranges reserved for private use (Co subcategory): 
    U+E000—U+F8FF (6,400 code points), U+F0000—U+FFFFD (65,534) and U+100000—U+10FFFD (65,534). 
    Surrogates (Cs subcategory) use the range U+D800—U+DFFF (2,048 code points).
    
    
    """

    ## Brute-force version: list all possible unicode ranges, but this list is not complete.
    #   text = re.sub('[\u0021-\u002f\u003a-\u0040\u005b-\u0060\u007b-\u007e\u00a1-\u00bf\u2000-\u206f\u2013-\u204a\u20a0-\u20bf\u2100-\u214f\u2150-\u218b\u2190-\u21ff\u2200-\u22ff\u2300-\u23ff\u2460-\u24ff\u2500-\u257f\u2580-\u259f\u25a0-\u25ff\u2600-\u26ff\u2e00-\u2e7f\u3000-\u303f\ufe50-\ufe6f\ufe30-\ufe4f\ufe10-\ufe1f\uff00-\uffef─◆╱]+','',text)

    text = ''.join(ch for ch in text
                   if unicodedata.category(ch)[0] not in ['P', 'S'])
    return text


## Remove numbers
def remove_numbers(text):
    return re.sub('\\d+', "", text)


## Remove alphabets
def remove_alphabets(text):
    return re.sub('[a-zA-Z]+', '', text)


## Combine every step
def normalize_corpus(corpus,
                     is_remove_extra_linebreaks=True,
                     is_remove_weird_chars=True,
                     is_seg=True,
                     is_remove_symbols=True,
                     is_remove_numbers=True,
                     is_remove_alphabets=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        if is_remove_extra_linebreaks:
            doc = remove_extra_linebreaks(doc)

        if is_remove_weird_chars:
            doc = remove_weird_chars(doc)

        if is_seg:
            doc = seg(doc)

        if is_remove_symbols:
            doc = remove_symbols(doc)

        if is_remove_alphabets:
            doc = remove_alphabets(doc)

        if is_remove_numbers:
            doc = remove_numbers(doc)

        normalized_corpus.append(remove_extra_spaces(doc))

    return normalized_corpus

## Extract an Article

- Grab the first article from Google news for demonstration

In [3]:
url = 'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRFptTXpJU0JYcG9MVlJYS0FBUAE?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
r = requests.get(url)
web_content = r.text
soup = BeautifulSoup(web_content, 'lxml')
title = soup.find_all('a', class_='DY5T1d')
first_art_link = title[0]['href'].replace('.', 'https://news.google.com', 1)

#print(first_art_link)
art_request = requests.get(first_art_link)
art_request.encoding = 'utf8'
soup_art = BeautifulSoup(art_request.text, 'lxml')

art_content = soup_art.find_all('p')
art_texts = [p.text for p in art_content]
print(art_texts)

['', '\r\n大陸中央電視台11日晚間「新聞聯播」報導，大陸國家安全機關破獲數百起台灣間諜情報機關竊密案件。隨後央視的「焦點訪談」專題報導，則以去年8月入境大陸後被捕的屏東縣枋寮鄉政顧問李孟居作為典型案例，李孟居除了在電視鏡頭前認罪，並指控其前往香港聲援反送中運動，是被現任枋寮鄉長陳亞麟「騙過去」。', '\n', '\r\n央視指稱，「去年香港持續數月的暴力示威中，『台獨』勢力的鬼影時而顯現。」李孟居除商人身分外，還是台獨組織「台灣聯合國協進會」理事，曾於去年8月赴香港執行「反送中加油」行動，並潛入深圳打探部隊集結情況，拍攝大量影音圖片向台獨組織發送報告。', '\n', '\r\n央視指出，2019年香港爆發反送中抗爭後，「台灣聯合國協進會」也立刻開始行動。李孟居的好友、「台灣聯合國協進會」候補理事陳亞麟再去年7月底找到李孟居，計畫去香港聲援黑衣人。陳亞麟另一身分是現任屏東縣枋寮鄉長。', '\n', '\r\n報導並公布李孟居與陳亞麟兩人的聊天記錄。二人曾商議，為配合蔡政府「支援香港民主運動」，受「台灣聯合國協進會」指使，兩人計畫入境香港實施「反送中加油」行動。', '\n', '\r\n聊天記錄還顯示，李孟居認為，此舉將日後可作為「政治獻禮」提升自身地位，有利於未來選立委。', '\n', '\r\n李孟居在電視認罪中供稱，他沒想到的是，臨行前，陳亞麟突然告訴他「沒有請到假，去不了了」。李孟居說，「那時候沒太多想，後來我才覺得不太對勁，是他早知道有危險，把我騙過去。」', '\n', '\r\n2019年8月18日，李孟居抵達香港，一落地就參與「反送中加油」行動，並第一時間將相關圖片發回台灣。隨後他從網上獲悉有武警在深圳集結的消息，於是第二天就進入深圳，並於當晚22時搭計程車到深圳灣體育場附近，但因天色已晚，沒有看到武警集結。', '\n', '\r\n報導稱，李孟居隨後在一棟高樓頂層的酒店裡確定了一個觀看武警集結的「最佳位置」。20日一早，李孟居起床後就搭計程車來到該酒店，並用隨身攜帶的DV和手機拍攝。隨後還下樓到深圳灣體育館一樓在警戒線附近拍攝，最終被捕。', '\n', '\r\n大陸已於去年10月31日對李孟居正式執行逮捕，並以涉嫌為境外刺探、非法提供秘密罪將案件移送檢察院審查起訴。報導稱，目前案件已進入法院審理程序。', '\n', '\r\

- Normalized results:

In [4]:
tnz.normalize_corpus([' '.join(art_texts)])

Building prefix dict from /Users/Alvin/GoogleDrive/_MySyncDrive/RepositoryData/data/jiaba/dict.txt.jiebatw.txt ...
Loading model from cache /var/folders/n7/ltpzwx813c599nfxfb94s_640000gn/T/jieba.u1b52b47246a0f2e6497af6bbe107adac.cache
Loading model cost 0.551 seconds.
Prefix dict has been built successfully.


['大陸 中央 電視台 日 晚間 新聞 聯播 報導 大陸 國家 安全 機關 破獲 數百 起 台灣 間諜 情報 機關 竊密 案件 隨後 央視 的 焦點 訪談 專題 報導 則以 去年 月 入境 大陸 後 被捕 的 屏東縣 枋寮鄉 政 顧問 李孟居 作為 典型 案例 李孟居 除了 在 電視 鏡頭 前 認罪 並 指控 其 前往 香港 聲援 反送 中 運動 是 被 現任 枋寮鄉 長陳 亞麟 騙 過去 央視 指稱 去年 香港 持續 數月 的 暴力 示威 中 台獨 勢力 的 鬼影 時而 顯現 李孟居除 商人 身分 外 還是 台獨 組織 台灣 聯合國 協進會 理事 曾 於 去年 月 赴 香港 執行 反送 中 加油 行動 並 潛入 深圳 打探 部隊 集結 情況 拍攝 大量 影音 圖片 向 台獨 組織 發送 報告 央視 指出 年 香港 爆發 反送 中 抗爭 後 台灣 聯合國 協進會 也 立刻 開始 行動 李孟居 的 好友 台灣 聯合國 協進會 候補 理事 陳 亞麟 再 去年 月底 找到 李孟居 計畫 去 香港 聲援 黑衣 人 陳 亞麟 另 一 身分 是 現任 屏東縣 枋寮鄉 長 報導 並 公布 李孟居 與 陳 亞麟 兩人 的 聊天 記錄 二人 曾 商議 為 配合 蔡 政府 支援 香港 民主 運動 受 台灣 聯合國 協進會 指使 兩人 計畫 入境 香港 實施 反送 中 加油 行動 聊天 記錄 還 顯示 李孟居 認為 此舉 將 日後 可 作為 政治 獻禮 提升 自身 地位 有利於 未來 選 立委 李孟居 在 電視 認罪 中 供稱 他 沒想到 的 是 臨行 前 陳 亞麟 突然 告訴 他 沒有 請到 假 去不了 了 李孟居 說 那 時候 沒 太多 想 後來 我 才 覺得 不 太 對勁 是 他 早 知道 有 危險 把 我 騙 過去 年 月 日 李孟居 抵達 香港 一 落地 就 參與 反送 中 加油 行動 並第 一時間 將 相關 圖片 發回 台灣 隨後 他 從 網上 獲悉 有 武警 在 深圳 集結 的 消息 於是 第二天 就 進入 深圳 並 於 當晚 時 搭 計程車 到 深圳 灣 體育場 附近 但 因 天色 已晚 沒有 看到 武警 集結 報導 稱 李孟居 隨後 在 一棟 高樓 頂層 的 酒店 裡 確定 了 一個 觀看 武警 集結 的 最佳 位置 日 一早 李孟居 起床 後 就 搭 計程車 來到 該